Importing all essential libraries


In [ ]:
!pip install faker
#Importing all libraries
import pandas as pd
import sqlite3
import numpy as np
import pickle as pickle
import subprocess,io
from time import time
from sklearn.decomposition import PCA
from itertools import repeat
from faker import Faker
from sklearn.utils import shuffle


#Optional Libraries
#from gensim.models import word2vec
#import gensim.downloader as api
#from gensim.models import KeyedVectors


#Glove pretrained word embeddings: https://nlp.stanford.edu/projects/glove/

In [ ]:
#Connect your drive as file system (If you have your files on drive)
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Word Embeddings part
(Pre-trained embeddings used: Glove)

---


Reducing the vector dimensions unsing PCA


In [ ]:
#This function is present in python program file, due to location issues I have maintained one copy here as well
#Reducing Glove word embeddings from 50 to 10 dimensions
Glove = {}
with io.open('/content/drive/My Drive/NLP_Data/glove.6B.50d.txt', encoding='utf8') as f:
#f = open('/content/drive/mydrive/glove.6B.50d.txt')

    print("Loading Glove vectors.")
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        Glove[word] = coefs
    f.close()

    print("Done.")
    X_train = []
    X_train_names = []
    for x in Glove:
            X_train.append(Glove[x])
            X_train_names.append(x)

    X_train = np.asarray(X_train)
    pca_embeddings = {}

# PCA to get Top Components
    pca =  PCA(n_components = 50)
    X_train = X_train - np.mean(X_train)
    X_fit = pca.fit_transform(X_train)
    U1 = pca.components_

    z = []

# Removing Projections on Top Components
    for i, x in enumerate(X_train):
	    for u in U1[0:7]:        
        	    x = x - np.dot(u.transpose(),x) * u 
	    z.append(x)

    z = np.asarray(z)

# PCA Dim Reduction
    pca =  PCA(n_components = 10)
    X_train = z - np.mean(z)
    X_new_final = pca.fit_transform(X_train)


# PCA to do Post-Processing Again
    pca =  PCA(n_components = 10)
    X_new = X_new_final - np.mean(X_new_final)
    X_new = pca.fit_transform(X_new)
    Ufit = pca.components_

    X_new_final = X_new_final - np.mean(X_new_final)

    final_pca_embeddings = {}
    embedding_file = open('/content/drive/My Drive/NLP_Data/pca_embed2.txt', 'w')

    for i, x in enumerate(X_train_names):
      final_pca_embeddings[x] = X_new_final[i]
      embedding_file.write("%s\t" % x)
      for u in Ufit[0:7]:
        final_pca_embeddings[x] = final_pca_embeddings[x] - np.dot(u.transpose(),final_pca_embeddings[x]) * u 

      for t in final_pca_embeddings[x]:
        embedding_file.write("%f\t" % t)
        
      embedding_file.write("\n")


    print("Reduced the dimensionality of the vector to 10 dimensions! \nPlease check pca_embed2.txt file")

In [ ]:
#This function is present in python program file, due to location issues I have maintained one copy here as well
#Function to get 10 dimensional vector from txt file
def get_vector(given_word):
  Glove = {}
  with io.open('/content/drive/My Drive/NLP_Data/pca_embed2.txt', encoding='utf8') as f:
  #f = open('/content/drive/My Drive/NLP_Data/pca_embed2.txt')

      #print("Loading Glove vectors.")
      for line in f:
          values = line.split()
          word = values[0]
          if word == given_word:
            coefs = np.asarray(values[1:], dtype='float32')
            given_word_vector = coefs
            break
  f.close()
  return given_word_vector

In [ ]:
#Getting Vectors for available posts
engineer_vector = get_vector('engineer')
manager_vector = get_vector('manager')
developer_vector = get_vector('developer')
ceo_vector = get_vector('ceo')
cto_vector = get_vector('cto')
coo_vector = get_vector('coo')
waiter_vector = get_vector('waiter')

#Getting Vectors for available cities
victoria_vector = get_vector('victoria')
vancouver_vector = get_vector('vancouver')
delhi_vector = get_vector('delhi')
pune_vector = get_vector('pune')
ottawa_vector = get_vector('ottawa')
toronto_vector = get_vector('toronto')
mumbai_vector = get_vector('mumbai')

#Stored these vectors in CSV Files
#File names:
#1. table1_name_post_city.csv
#2. table2_vectors_for_post_city.csv
#3. table3_vectors_for_posts.csv
#4. table4_vectors_for_cities.csv

In [ ]:
#This function is present in python program file, due to location issues I have maintained one copy here as well

#Defining Cosine Similarity Function
def cos_sim(a, b):
    """Takes 2 vectors a, b and returns the cosine similarity according 
    to the definition of the dot product
    """
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    return dot_product / (norm_a * norm_b)

#Testing the function
similarity = cos_sim(engineer_vector,developer_vector)
print("Similarity between Engineer and Developer:",similarity)

Similarity between Engineer and Developer: 0.79248005


Database creation part

---



In [ ]:
#This function is present in python program file, due to location issues I have maintained one copy here as well

#Generating Data
def get_data(n,j):
  #Getting fake names
  fake = Faker()
  name_df = [fake.name() for i in range(n)]
  name_df = pd.DataFrame((name_df),columns=['name'])

  #Dataframe with Vectors for respective posts and post
  data = pd.read_csv (r'/content/drive/My Drive/NLP_Data/table3_vectors_for_posts.csv')   
  orig_post_df = pd.DataFrame(data, columns= ['pi1','pi2','pi3','pi4','pi5','pi6','pi7','pi8','pi9','pi10','post'])
  post_df = orig_post_df.iloc[:,:]
  #print(df3)

  #Dataframe with Vectors for respective cities and city
  data = pd.read_csv (r'/content/drive/My Drive/NLP_Data/table4_vectors_for_cities.csv')   
  orig_city_df = pd.DataFrame(data, columns= ['ci1','ci2','ci3','ci4','ci5','ci6','ci7','ci8','ci9','ci10','city'])
  city_df = orig_city_df.iloc[:,:]
  #print(df4)

    #Shuffling and storing the data n times
  new_city_df = city_df
  new_post_df = post_df

  for i in range(int(n/7)):
    city_df = shuffle(city_df)
    post_df = shuffle(post_df)
    new_city_df = pd.concat([new_city_df, city_df], axis=0)
    new_post_df = pd.concat([new_post_df, post_df], axis=0)

  #Getting n number of data
  normal_city_df = new_city_df.iloc[:,10]
  normal_post_df = new_post_df.iloc[:,10]
  n_city_df = new_city_df.iloc[:,:-1]
  n_post_df = new_post_df.iloc[:,:-1]

  n_city_df= new_city_df.head(n)
  n_post_df= new_post_df.head(n)
  normal_city_df= normal_city_df.head(n)
  normal_post_df= normal_post_df.head(n)

  #Resetting index
  n_city_df.reset_index(drop=True, inplace=True)
  n_post_df.reset_index(drop=True, inplace=True)
  normal_city_df.reset_index(drop=True, inplace=True)
  normal_post_df.reset_index(drop=True, inplace=True)

  #Dataframe ready for table 1 with name, post and city word embeddings
  df1 = pd.concat([name_df,n_post_df,n_city_df], axis=1)
  normal_df1 = pd.concat([name_df,normal_post_df,normal_city_df], axis=1)
  #print(df1)

  #Shuffling and resetting index for table 2 
  new_city_df = shuffle(new_city_df)
  new_post_df = shuffle(new_post_df)

  normal2_city_df = new_city_df.iloc[:,10]
  normal2_post_df = new_post_df.iloc[:,10]
  new_city_df = new_city_df.iloc[:,:-1]
  new_post_df = new_post_df.iloc[:,:-1]

  new_city_df= new_city_df.head(j)
  new_post_df= new_post_df.head(j)
  normal2_city_df= normal_city_df.head(j)
  normal2_post_df= normal_post_df.head(j)

  new_city_df.reset_index(drop=True, inplace=True)
  new_post_df.reset_index(drop=True, inplace=True)

  #Dataframe ready for table 2 with post and city
  df2 = pd.concat([new_post_df,new_city_df], axis=1)
  normal_df2 = pd.concat([normal2_post_df,normal2_city_df], axis=1)
  #print(df2)

  #Creating database
  connection = sqlite3.connect("position_city_database_with_embeddings.db") 
  crsr = connection.cursor() 

  #Comment the table creation and insertion of data into the table if the database is already created once.
  #Creating table1 with name, embeddings of post, and embeddings of city
  crsr.execute('CREATE TABLE name_post_city (NAME nvarchar(50),pi1 float,pi2 float,pi3 float,pi4 float,pi5 float,pi6 float,pi7 float,pi8 float,pi9 float,pi10 float, ci1 float,ci2 float,ci3 float,ci4 float,ci5 float,ci6 float,ci7 float,ci8 float,ci9 float,ci10 float, FOREIGN KEY (ci1,ci2,ci3,ci4,ci5,ci6,ci7,ci8,ci9,ci10) REFERENCES em_city_name(ci1,ci2,ci3,ci4,ci5,ci6,ci7,ci8,ci9,ci10), FOREIGN KEY (pi1,pi2,pi3,pi4,pi5,pi6,pi7,pi8,pi9,pi10) REFERENCES em_post_city(pi1,pi2,pi3,pi4,pi5,pi6,pi7,pi8,pi9,pi10))')
  df1.to_sql('name_post_city', connection, if_exists='replace', index = False)
  crsr.execute('''SELECT * FROM name_post_city''')
  print("Table 1: Name_Post_City Data")
  for row in crsr.fetchall():
      print (row)

  #Creating normal table1 with name,post, and city
  crsr.execute('CREATE TABLE normal_name_post_city (NAME nvarchar(50),pi1 float,pi2 float,pi3 float,pi4 float,pi5 float,pi6 float,pi7 float,pi8 float,pi9 float,pi10 float, ci1 float,ci2 float,ci3 float,ci4 float,ci5 float,ci6 float,ci7 float,ci8 float,ci9 float,ci10 float, FOREIGN KEY (ci1,ci2,ci3,ci4,ci5,ci6,ci7,ci8,ci9,ci10) REFERENCES em_city_name(ci1,ci2,ci3,ci4,ci5,ci6,ci7,ci8,ci9,ci10), FOREIGN KEY (pi1,pi2,pi3,pi4,pi5,pi6,pi7,pi8,pi9,pi10) REFERENCES em_post_city(pi1,pi2,pi3,pi4,pi5,pi6,pi7,pi8,pi9,pi10))')
  normal_df1.to_sql('normal_name_post_city', connection, if_exists='replace', index = False)
  crsr.execute('''SELECT * FROM normal_name_post_city''')
  print("Normal Table 1: Normal_Name_Post_City Data")
  for row in crsr.fetchall():
      print (row)

  #Creating table2 with embeddings of post and embeddings of city
  crsr.execute('CREATE TABLE post_city (pi1 float,pi2 float,pi3 float,pi4 float,pi5 float,pi6 float,pi7 float,pi8 float,pi9 float,pi10 float, ci1 float,ci2 float,ci3 float,ci4 float,ci5 float,ci6 float,ci7 float,ci8 float,ci9 float,ci10 float, FOREIGN KEY (ci1,ci2,ci3,ci4,ci5,ci6,ci7,ci8,ci9,ci10) REFERENCES em_city_name(ci1,ci2,ci3,ci4,ci5,ci6,ci7,ci8,ci9,ci10), FOREIGN KEY (pi1,pi2,pi3,pi4,pi5,pi6,pi7,pi8,pi9,pi10) REFERENCES em_post_city(pi1,pi2,pi3,pi4,pi5,pi6,pi7,pi8,pi9,pi10))')
  df2.to_sql('post_city', connection, if_exists='replace', index = False)
  print("\nTable 2: Post_City Data")
  crsr.execute('''SELECT * FROM post_city''')
  for row in crsr.fetchall():
      print (row)
  

  #Creating normal table2 with post and city
  crsr.execute('CREATE TABLE normal_post_city (pi1 float,pi2 float,pi3 float,pi4 float,pi5 float,pi6 float,pi7 float,pi8 float,pi9 float,pi10 float, ci1 float,ci2 float,ci3 float,ci4 float,ci5 float,ci6 float,ci7 float,ci8 float,ci9 float,ci10 float, FOREIGN KEY (ci1,ci2,ci3,ci4,ci5,ci6,ci7,ci8,ci9,ci10) REFERENCES em_city_name(ci1,ci2,ci3,ci4,ci5,ci6,ci7,ci8,ci9,ci10), FOREIGN KEY (pi1,pi2,pi3,pi4,pi5,pi6,pi7,pi8,pi9,pi10) REFERENCES em_post_city(pi1,pi2,pi3,pi4,pi5,pi6,pi7,pi8,pi9,pi10))')
  normal_df2.to_sql('normal_post_city', connection, if_exists='replace', index = False)
  print("\nNormal Table 2: Normal_Post_City Data")
  crsr.execute('''SELECT * FROM normal_post_city''')
  for row in crsr.fetchall():
      print (row)

  #Creating table3 with embeddings of post and name of posts
  crsr.execute('CREATE TABLE em_post_name (pi1 float,pi2 float,pi3 float,pi4 float,pi5 float,pi6 float,pi7 float,pi8 float,pi9 float,pi10 float, post nvarchar(50), PRIMARY KEY(pi1,pi2,pi3,pi4,pi5,pi6,pi7,pi8,pi9,pi10))')
  orig_post_df.to_sql('em_post_name', connection, if_exists='replace', index = False)
  print("\nTable 3: Em_Post_Name Data")
  crsr.execute('''SELECT * FROM em_post_name''')
  for row in crsr.fetchall():
      print (row)

  #Creating table4 with embeddings of city and name of cities
  crsr.execute('CREATE TABLE em_city_name (ci1 float,ci2 float,ci3 float,ci4 float,ci5 float,ci6 float,ci7 float,ci8 float,ci9 float,ci10 float, city nvarchar(50), PRIMARY KEY(ci1,ci2,ci3,ci4,ci5,ci6,ci7,ci8,ci9,ci10))')
  orig_city_df.to_sql('em_city_name', connection, if_exists='replace', index = False)
  print("\nTable 4: Em_City_Name Data")
  crsr.execute('''SELECT * FROM em_city_name''')
  for row in crsr.fetchall():
      print (row)

  connection.commit()

  #Execute the following line to print all the SQL queries when executed
  #connection.set_trace_callback(print)

  #Incase you want to drop all tables:
  #crsr.execute('DROP TABLE name_post_city')
  #crsr.execute('DROP TABLE post_city')
  #crsr.execute('DROP TABLE em_post_name')
  #crsr.execute('DROP TABLE em_city_name')
  #connection.commit()  

get_data(100,50)

Table 1: Name_Post_City Data
('Robin Vaughn', 0.0, -1e-06, 1e-06, 0.0, -1e-06, 6e-06, 1.2e-05, 0.32, -0.31, 0.966, 'engineer', 0.0, 0.0, 1e-06, -1e-06, -7e-06, 5e-06, 1.2e-05, 0.457, 1.49, 1.24, 'victoria')
('Jill Brown', 0.0, -1e-06, 1e-06, 0.0, -4e-06, 5e-06, 1.4999999999999999e-05, 0.478, 0.431, 1.07, 'developer', 0.0, 1e-06, 0.0, -1e-06, -4e-06, -1e-06, 3e-06, 0.215, 1.22, 0.192, 'vancouver')
('Brenda Mendez', 0.0, -1e-06, 0.0, 0.0, -1e-06, 1.1e-05, -4e-06, -0.252, 0.00328, 1.39, 'manager', 0.0, -1e-06, 0.0, 0.0, -2e-06, 9e-06, 1e-06, -0.0251, 0.397, 1.26, 'mumbai')
('Courtney Aguilar', 0.0, -1e-06, 0.0, 0.0, 2e-06, 6e-06, -9e-06, -0.424, -0.6579999999999999, 0.485, 'coo', 0.0, -2e-06, 1e-06, 0.0, 0.0, 8e-06, 2e-06, -0.0689, -0.315, 1.06, 'delhi')
('Robert Hunter', 0.0, 0.0, -2e-06, 0.0, -1e-06, 6e-06, -2.2e-05, -0.736, 0.625, 0.428, 'cto', 0.0, -1e-06, 0.0, 0.0, 0.0, 8e-06, -3e-06, -0.205, -0.23600000000000002, 0.95, 'pune')
('Joshua Mendez', 0.0, -2e-06, 2e-06, 0.0, -3e-06, 5e-06

Function for calculating cosine similarity on:

---
1. Posts (Engineer, Developer, etc)

In [ ]:
connection = sqlite3.connect("position_city_database_with_embeddings.db") 
crsr = connection.cursor() 

#This function is present in python program file, due to location issues I have maintained one copy here as well
def sel_func_post(q,threshold,list_type):
  crsr.execute("select pi1,pi2,pi3,pi4,pi5,pi6,pi7,pi8,pi9,pi10 from em_post_name where post ='%s'" %q)
  q_vect = crsr.fetchall()
  crsr.execute("select pi1,pi2,pi3,pi4,pi5,pi6,pi7,pi8,pi9,pi10,post from em_post_name")
  t_vect = crsr.fetchall()
  vect_t = [tuple(list(x)[0:10]) for x in t_vect] # Getting all vectors in em_post_name
  vect_names = [list(x).pop(-1) for x in t_vect] # Getting all the corresponding post names to the vectors
  #print(vect_t) # Used to print all vectors in em_post_name
  #print(vect_names) # Used to print all the corresponding posts to the vectors
  similarity_array = [cos_sim(q_vect, x)[0] for x in vect_t] # Getting similarity scores for all vectors in table
  #print(similarity_array) # Used to print similary between vectors
  a = np.array(similarity_array)
  index = np.where(a > threshold)[0] # Getting index of all the post which are having similar value > threshold
  #print(index) # Printing indexes 
  new_vect_t = [vect_t[x] for x in index] # Getting new vectors according to those indexes
  if list_type==1:
    vect_name_list = [vect_names[i] for i in index] 
    return vect_name_list #This is to return the name of the cities it found to be similar
  #print(new_vect_t) # Printing the new vector
  #print(list(np.array(vect_t)[index][0])) # Checking the element
  join_list = []
  for x in range(0,len(new_vect_t[0])):
    join_list.append(tuple([i[x] for i in new_vect_t])) # Joining the new vectors together in a list
  #print(join_list) # To print the join_list
  return join_list # Returning the join_list

# Executing the function

t = 0.85 # Defining threshold (You can change according to your requirements)
sel_func_post('ceo',t,0)

[(0.0, 0.0, 0.0, 0.0),
 (-1e-06, -1e-06, -2e-06, -2e-06),
 (1e-06, 1e-06, 2e-06, 2e-06),
 (0.0, 0.0, 0.0, 0.0),
 (-1e-06, -4e-06, -3e-06, -1e-06),
 (6e-06, 5e-06, 5e-06, 8e-06),
 (1.2e-05, 1.4999999999999999e-05, 2.4e-05, 1.2e-05),
 (0.32, 0.478, 0.733, 0.28),
 (-0.31, 0.431, -0.16899999999999998, -0.44),
 (0.966, 1.07, 1.06, 1.17)]

Function for calculating cosine similarity on:

---
2. Cities (Victoria, Pune, etc)

In [ ]:
#This function is present in python program file, due to location issues I have maintained one copy here as well
connection = sqlite3.connect("position_city_database_with_embeddings.db") 
crsr = connection.cursor() 
def sel_func_city(q,threshold,list_type):
  crsr.execute("select ci1,ci2,ci3,ci4,ci5,ci6,ci7,ci8,ci9,ci10 from em_city_name where city ='%s'" %q)
  q_vect = crsr.fetchall()
  crsr.execute("select ci1,ci2,ci3,ci4,ci5,ci6,ci7,ci8,ci9,ci10,city from em_city_name")
  t_vect = crsr.fetchall()
  vect_t = [tuple(list(x)[0:10]) for x in t_vect] # Getting all vectors in em_city_name
  vect_names = [list(x).pop(-1) for x in t_vect] # Getting all the corresponding city names to the vectors
  #print(vect_t) # Used to print all vectors in em_post_name
  #print(vect_names) # Used to print all the corresponding posts to the vectors
  similarity_array = [cos_sim(q_vect, x)[0] for x in vect_t] # Getting similarity scores for all vectors in table
  #print(similarity_array) # Used to print similary between vectors
  a = np.array(similarity_array) 
  index = np.where(a > threshold)[0] # Getting index of all the post which are having similar value > threshold
  #print(index) # Printing indexes 
  new_vect_t = [vect_t[x] for x in index] # Getting new vectors according to those indexes
  if list_type == 1:
    vect_name_list = [vect_names[i] for i in index] 
    return vect_name_list #This is to return the name of the cities it found to be similar
  #print(new_vect_t) # Printing the new vector
  #print(list(np.array(vect_t)[index][0])) # Checking the element
  join_list = []
  for x in range(0,len(new_vect_t[0])):
    join_list.append(tuple([i[x] for i in new_vect_t]))  # Joining the new vectors together in a list
  #print(join_list) # To print the join_list
  return join_list # Returning the join_list

# Executing the function

t = 0.82 # Defining threshold (You can change according to your requirements)
sel_func_city('vancouver',t,0)

[(0.0, 0.0, 0.0, 0.0),
 (0.0, 1e-06, 1e-06, 1e-06),
 (1e-06, 0.0, -1e-06, 0.0),
 (-1e-06, -1e-06, -1e-06, -1e-06),
 (-7e-06, -4e-06, -2e-06, -5e-06),
 (5e-06, -1e-06, 1e-06, 0.0),
 (1.2e-05, 3e-06, -1.1e-05, 7e-06),
 (0.457, 0.215, -0.301, 0.324),
 (1.49, 1.22, 0.96, 1.28),
 (1.24, 0.192, 0.179, 0.47200000000000003)]

Joining two tables according to word cosine similarities

---


In [ ]:
connection = sqlite3.connect("position_city_database_with_embeddings.db") 
crsr = connection.cursor() 
def makeqmarks(i):
    return ', '.join(repeat('?', i))

#placeholder = '?'
#format_strings = ','.join(['%s'] * len(join_list[0]))


def join_table_fun_city(join_list):

# ************* Join table query *************
  queryc = 'SELECT b.NAME, d.post, c.city \
  FROM em_post_name d, em_city_name c, post_city a \
  INNER JOIN name_post_city b \
  ON a.ci1 = b.ci1 AND \
  a.ci2 = b.ci2 AND \
  a.ci3 = b.ci3 AND \
  a.ci4 = b.ci4 AND \
  a.ci5 = b.ci5 AND \
  a.ci6 = b.ci6 AND \
  a.ci7 = b.ci7 AND \
  a.ci8 = b.ci8 AND \
  a.ci9 = b.ci9 AND \
  a.ci10 = b.ci10 \
  WHERE \
  a.ci1 = c.ci1 AND \
  a.ci2 = c.ci2 AND \
  a.ci3 = c.ci3 AND \
  a.ci4 = c.ci4 AND \
  a.ci5 = c.ci5 AND \
  a.ci6 = c.ci6 AND \
  a.ci7 = c.ci7 AND \
  a.ci8 = c.ci8 AND \
  a.ci9 = c.ci9 AND \
  a.ci10 = c.ci10 AND \
  a.pi1 = d.pi1 AND \
  a.pi2 = d.pi2 AND \
  a.pi3 = d.pi3 AND \
  a.pi4 = d.pi4 AND \
  a.pi5 = d.pi5 AND \
  a.pi6 = d.pi6 AND \
  a.pi7 = d.pi7 AND \
  a.pi8 = d.pi8 AND \
  a.pi9 = d.pi9 AND \
  a.pi10 = d.pi10 AND \
  a.ci1 IN (%s) AND \
  a.ci2 IN (%s) AND \
  a.ci3 IN (%s) AND \
  a.ci4 IN (%s) AND \
  a.ci5 IN (%s) AND \
  a.ci6 IN (%s) AND \
  a.ci7 IN (%s) AND \
  a.ci8 IN (%s) AND \
  a.ci9 IN (%s) AND \
  a.ci10 IN (%s);' \
  % (makeqmarks(len(join_list[0])), makeqmarks(len(join_list[1])), makeqmarks(len(join_list[2])),makeqmarks(len(join_list[3])),makeqmarks(len(join_list[4])),makeqmarks(len(join_list[5])),makeqmarks(len(join_list[6])),makeqmarks(len(join_list[7])),makeqmarks(len(join_list[8])),makeqmarks(len(join_list[9])))
  #print(join_list[0]) # To check the elements in 0th index
  return queryc

def join_table_fun_post(join_list):

# ************* Join table query *************
  queryp = 'SELECT b.NAME, d.post, c.city \
  FROM em_post_name d, em_city_name c, post_city a \
  INNER JOIN name_post_city b \
  ON a.ci1 = b.ci1 AND \
  a.pi2 = b.pi2 AND \
  a.pi3 = b.pi3 AND \
  a.pi4 = b.pi4 AND \
  a.pi5 = b.pi5 AND \
  a.pi6 = b.pi6 AND \
  a.pi7 = b.pi7 AND \
  a.pi8 = b.pi8 AND \
  a.pi9 = b.pi9 AND \
  a.pi10 = b.pi10 \
  WHERE \
  a.ci1 = c.ci1 AND \
  a.ci2 = c.ci2 AND \
  a.ci3 = c.ci3 AND \
  a.ci4 = c.ci4 AND \
  a.ci5 = c.ci5 AND \
  a.ci6 = c.ci6 AND \
  a.ci7 = c.ci7 AND \
  a.ci8 = c.ci8 AND \
  a.ci9 = c.ci9 AND \
  a.ci10 = c.ci10 AND \
  a.pi1 = d.pi1 AND \
  a.pi2 = d.pi2 AND \
  a.pi3 = d.pi3 AND \
  a.pi4 = d.pi4 AND \
  a.pi5 = d.pi5 AND \
  a.pi6 = d.pi6 AND \
  a.pi7 = d.pi7 AND \
  a.pi8 = d.pi8 AND \
  a.pi9 = d.pi9 AND \
  a.pi10 = d.pi10 AND \
  a.pi1 IN (%s) AND \
  a.pi2 IN (%s) AND \
  a.pi3 IN (%s) AND \
  a.pi4 IN (%s) AND \
  a.pi5 IN (%s) AND \
  a.pi6 IN (%s) AND \
  a.pi7 IN (%s) AND \
  a.pi8 IN (%s) AND \
  a.pi9 IN (%s) AND \
  a.pi10 IN (%s);' \
  % (makeqmarks(len(join_list[0])), makeqmarks(len(join_list[1])), makeqmarks(len(join_list[2])),makeqmarks(len(join_list[3])),makeqmarks(len(join_list[4])),makeqmarks(len(join_list[5])),makeqmarks(len(join_list[6])),makeqmarks(len(join_list[7])),makeqmarks(len(join_list[8])),makeqmarks(len(join_list[9])))
  #print(join_list[0]) # To check the elements in 0th index

  return queryp


Executing the SQL join query and displaying the time taken to execute the query

In [ ]:
# ************* Getting Joined List and Calling the function *************

join_list = sel_func_city('vancouver', 0.82,0) # Getting locations which having similarity score of more than .85 with 'Victoria'
queryc = join_table_fun_city(join_list)
#join_list = sel_func_post('engineer', 0.82,0)
#queryp = join_table_fun_post(join_list)
#print(join_list)


# ************* Executing and Displaying the Query *************

#Getting the execution time
tic = time()
#Query for city
crsr.execute(queryc, join_list[0] + join_list[1] + join_list[2] + join_list[3] + join_list[4] + join_list[5] + join_list[6] + join_list[7] + join_list[8] + join_list[9])
#Query for post
#crsr.execute(queryp, join_list[0] + join_list[1] + join_list[2] + join_list[3] + join_list[4] + join_list[5] + join_list[6] + join_list[7] + join_list[8] + join_list[9])
toc = time()
ot = toc-tic

#To print the query output
#for row in crsr.fetchall():
#    print (row)
print("Execution time: ",(ot)) 

# Actual Generated Query
# SELECT b.name, c.city, d.post FROM em_city_name c, em_post_name d, post_city a INNER JOIN name_post_city b ON a.ci1 = b.ci1 AND a.ci2 = b.ci2 AND a.ci3 = b.ci3 AND a.ci4 = b.ci4 AND a.ci5 = b.ci5 AND a.ci6 = b.ci6 AND a.ci7 = b.ci7 AND a.ci8 = b.ci8 AND a.ci9 = b.ci9 AND a.ci10 = b.ci10 WHERE a.ci1 IN (0.0, 0.0, 0.0) AND a.ci2 IN (0.0, 1.0e-06, 1.0e-06) AND a.ci3 IN (1.0e-06, 0.0, 0.0) AND a.ci4 IN (-1.0e-06, -1.0e-06, -1.0e-06) AND a.ci5 IN (-7.0e-06, -4.0e-06, -5.0e-06) AND a.ci6 IN (5.0e-06, -1.0e-06, 0.0) AND a.ci7 IN (1.2e-05, 3.0e-06, 7.0e-06) AND a.ci8 IN (0.457, 0.215, 0.324) AND a.ci9 IN (1.49, 1.22, 1.28) AND a.ci10 IN (1.24, 0.192, 0.472);

Execution time:  0.0023317337036132812


Evaluating the Quality(Accuracy)

---

In [ ]:
#Checking the model
def check_accuracy(join_list, expected_output):
  if join_list == expected_output:
    a = (len(join_list) / len(expected_output))*100
    print("Accuracy =",a)
  else:
    difference = set(expected_output).symmetric_difference(set(join_list))
    list_difference = list(difference)
    #print(list_difference)
    intersection = set(expected_output).intersection(set(join_list))
    list_similarity = list(intersection)
    #print(list_similarity)
    out_off = len(list_difference) + len(list_similarity)
    #print(out_off)
    #print(list_similarity)
    a = (len(list_similarity) / out_off)*100
    print("Accuracy =",a,"%")

t = 0.82 #Threshold is set to 0.82

#List for cities
join_list = sel_func_city('vancouver', t,1)
expected_output = ['victoria', 'vancouver', 'ottawa', 'toronto']

#List for posts
#join_list = sel_func_post('engineer',t,1)
#expected_output = ['engineer', 'developer']

#Calling the function
check_accuracy(join_list,expected_output)

Accuracy = 100.0


Calculating the execution time of the query with the regular approach

---

In [ ]:
#Calculating execution time for regular query
connection = sqlite3.connect("position_city_database_with_embeddings.db") 
crsr = connection.cursor() 

#Getting the execution time
tick = time()
#Change the expected ouput list with the IN parameters in the query

#Query for post
#crsr.execute("SELECT b.NAME, a.post, a.city FROM  normal_post_city a INNER JOIN normal_name_post_city b ON a.post = b.post where a.post IN ('engineer','developer')")

#Query for city
crsr.execute("SELECT b.NAME, a.post, a.city FROM  normal_post_city a INNER JOIN normal_name_post_city b ON a.city = b.city where a.city IN ('victoria', 'vancouver', 'ottawa', 'toronto')")

tock = time()
ora = tock-tick

#To print the query output
#for row in crsr.fetchall():
#    print (row) 
print("Regular Approach Execution time: ",(ora)) 


Regular Approach Execution time:  0.0006308555603027344


Comparing the execution time of the query with the regular approach

---

In [ ]:
#Execution Time difference
t = ot - ora
print("Execution time difference between our approach and regular approach: ",t)

Execution time difference between our approach and regular approach:  0.0017008781433105469


In [ ]:
def join_table_city(join_list):

# ************* Join table query *************
  queryc = 'SELECT b.NAME, a.post, a.city \
  FROM normal_post_city a \
  INNER JOIN normal_name_post_city b \
  ON a.city = b.city \
  WHERE \
  a.city IN (%s);' \
  % (makeqmarks(len(join_list)))
  #print(join_list[0]) # To check the elements in 0th index
  return queryc

def join_table_post(join_list):

# ************* Join table query *************
  queryp = 'SELECT b.NAME, a.post, a.city \
  FROM normal_post_city a \
  INNER JOIN normal_name_post_city b \
  ON a.post = b.post \
  WHERE \
  a.post IN (%s);' \
  % (makeqmarks(len(join_list)))
  #print(join_list[0]) # To check the elements in 0th index
  return queryp

In [ ]:
#join_list = sel_func_post('engineer',t,1)
#expected_output_post = ['engineer', 'developer']
expected_output_city = ['victoria', 'vancouver', 'ottawa', 'toronto']
queryci = join_table_city(expected_output_city)
#Query for city
tick = time()
crsr.execute(queryci, expected_output_city)
# crsr.execute("SELECT b.NAME, a.post, a.city FROM  normal_post_city a INNER JOIN normal_name_post_city b ON a.city = b.city where a.city IN ('victoria', 'vancouver', 'ottawa', 'toronto')")
tock = time()
ora = tock-tick
# #To print the query output
# for row in crsr.fetchall():
#     print (row) 
print("Regular Approach Execution time: ",ora) 

Regular Approach Execution time:  0.00064849853515625


In [ ]:
#city_list = ['victoria','vancouver','delhi','pune','ottawa','toronto','mumbai']
city_list = ['engineer','manager','developer','ceo','cto','coo','waiter']
expected_output_city = ['engineer', 'developer']
#Calculating Accuracy
join_list1 = sel_func_post('engineer', 0.82,1)
print(join_list1)
intersection = set(expected_output_city).intersection(set(join_list1))
list_similarity = list(intersection)
print(list_similarity)
difference = set(expected_output_city).symmetric_difference(set(join_list1))
list_difference = list(difference)

if (len(list_difference)+len(expected_output_city))!=0:
  accuracy = len(list_similarity)/(len(list_difference)+len(expected_output_city))
else:
  accuracy =1
print(accuracy)

#Calculating Precision
diff = set(city_list).symmetric_difference(set(expected_output_city))
list_diff = list(diff)

inter_correct = set(join_list1).intersection(set(list_diff))
list_inter_correct = list(inter_correct)
print(len(list_difference))
if len(list_difference)!=0:
  precision = len(list_inter_correct)/len(list_difference)
else:
  precision =1
print(precision)

#Calculating Recall
print(len(list_inter_correct))
if len(list_diff)!=0 and len(list_inter_correct)!=0:
  recall = len(list_inter_correct)/len(list_diff)
else:
  recall =1 
print(recall)
#Calculating F1 Score
F1 = 2 * (precision * recall) / (precision + recall)
print(F1)


['engineer', 'manager', 'ceo', 'waiter']
['engineer']
0.16666666666666666
4
0.75
3
0.6
0.6666666666666665
